# Cruzamento de dados para o Município de São Paulo

## Importações

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import os
import geopandas as gpd
import pandas as pd

#from shapely import LineString, Polygon, MultiPolygon, distance, intersects, minimum_bounding_radius as min_radius
#from shapely.geometry import box
#from shapely.wkt import loads, dumps

In [2]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Mounted at /content/drive


In [3]:
nome_dir = 'PIC1022-SQ-MSP-Dados'
if not os.path.isdir(nome_dir):
    os.mkdir(nome_dir)

In [4]:
UTMCRS = 'EPSG:31983'

### Leitura dos arquivos base

In [ ]:
gdf = gpd.read_file('PIC1022-SSSQQQ-MSP/perimetros_compativeis_agg.gpkg')
df_A = pd.read_csv('PIC1022-SSSQQQ-MSP/matriz_compat_PIC1022.csv', sep='\t', dtype={'CD_PERIMETRO':str, 'CD_PICSQ':str})
df_B = pd.read_csv('PIC1022-SSSQQQ-MSP/matriz_compat_SSSQQQ.csv', sep='\t', dtype={'SSSQQQ':str, 'CD_PICSQ':str})

## 1. Agregação com mancha urbana, expansão urbana, Embraesp, AGSN 2010

In [ ]:
# Juntar de arquivo já agregado
agg_gdf = gpd.read_file('2010-2022-RMSP-Dados/PICs_RMSP_1022_agg.gpkg')

In [ ]:
agg_gdf['CD_MUN'] = agg_gdf['CD_PERIMETRO'].apply(lambda x: x[:7])
agg_gdf['CD_DIST'] = agg_gdf['CD_PERIMETRO'].apply(lambda x: x[:11])
agg_gdf = agg_gdf.query('CD_MUN == "3550308"')

In [ ]:
cols = {
    'DOM_COL_2022': 'sum',
    'DOM_PAR_2022': 'sum',
    'DOM_PI_2010': 'sum',
    'DOM_PO_2010': 'sum',
    'DOM_PO_2022': 'sum',
    'DOM_PO_IMPUT_2022': 'sum',
    'DOM_PP_2010': 'sum',
    'DOM_TOT_2010': 'sum',
    'DOM_TOT_2022': 'sum',
    'POP_DOM_PO_2022': 'sum',
    'POP_DOM_PP_2010': 'sum',
    'POP_TOT_2010': 'sum',
    'POP_TOT_2022': 'sum',
    'AREA_HA': 'sum',
    'AREA_URB_HA_2010': 'sum',
    'AREA_URB_HA_2022': 'sum',
    'EXPANSAO_HA_COND_HOR': 'sum',
    'EXPANSAO_HA_CONJ_VERT': 'sum',
    'EXPANSAO_HA_FAV_OCUP_LOT': 'sum',
    'EXPANSAO_HA_IND_GALP': 'sum',
    'EXPANSAO_HA_HOR': 'sum',
    'EXPANSAO_HA_OUTRO': 'sum',
    'EXPANSAO_HA_RODOVIA': 'sum',
    'EXPANSAO_HA_VERT': 'sum',
    'AREA_EXPANSAO_HA': 'sum',
    'AU_EMP_10A22': 'sum',
    'GAR_10A22': 'sum',
    'TT_UNID_10A22': 'sum',
    'AGSN_2010': 'max'
}

In [ ]:
agg_data = agg_gdf.merge(df_A, on='CD_PERIMETRO', how='left')
agg_data = agg_data.pivot_table(index='CD_PICSQ', values=[i for i in cols.keys()], aggfunc=cols).reset_index()
agg_data = agg_data.merge(gdf[['CD_PICSQ', 'geometry']], on='CD_PICSQ', how='left')
agg_gdf = gpd.GeoDataFrame(agg_data, geometry='geometry', crs=UTMCRS)

In [ ]:
# Densidade populacional
for year in ['2010', '2022']:
    agg_gdf[f'dens_pop_tot_ha_{year}'] = agg_gdf[f'POP_TOT_{year}']/(agg_gdf['geometry'].area/10000)

    # Densidade domiciliar (total)
    agg_gdf[f'dens_dom_tot_ha_{year}'] = agg_gdf[f'DOM_TOT_{year}']/(agg_gdf['geometry'].area/10000)

    # Densidade domiciliar (ocupados)
    agg_gdf[f'dens_dom_po_ha_{year}'] = agg_gdf[f'DOM_PO_{year}']/(agg_gdf['geometry'].area/10000)

    # Média de moradores por domicílio
    if year != '2022':
        agg_gdf[f'med_pop_dom_pp_{year}'] = agg_gdf[f'POP_DOM_PP_{year}']/agg_gdf[f'DOM_PP_{year}']
    else:
        agg_gdf[f'med_pop_dom_po_{year}'] = agg_gdf[f'POP_DOM_PO_{year}']/agg_gdf[f'DOM_PO_{year}']

## 2. Agregação com dados IPTU

In [ ]:
iptu = pd.read_csv('../iptu_10_22_variacoes_final.csv', sep=';', dtype={'SSSQQQ':str}, decimal=',')

# Merge e reagregação
aggcols = [i for i in iptu.columns if i != 'SSSQQQ']
for col in aggcols:
    iptu[col] = iptu[col].astype(float)

df_iptu = iptu.merge(df_B, on='SSSQQQ', how='left')[['CD_PICSQ']+aggcols].fillna(0)
df_iptu = df_iptu.pivot_table(index='CD_PICSQ', values=aggcols, aggfunc='sum').reset_index()

# Renomear colunas
renamedic = {k:k[:-2]+'20'+k[-2:] for k in df_iptu.columns if k.split('_')[-1] in ['00','10','22']}
df_iptu = df_iptu.rename(columns=renamedic)

In [ ]:
# Exportação
#cols = ['CD_PCSQ','iptu_var_area_res','iptu_var_area_terr_res','iptu_var_unid_res'] + list(renamedic.values())
#df_iptu = df_iptu[cols]
df_iptu.to_csv(f'{nome_dir}/iptu.csv', index=False, sep='\t', decimal=',')

## 3. Agregação com dados de licenciamento

In [ ]:
sel = pd.read_csv('../SQ_alvaras_residenciais.csv', sep='\t', decimal=',', dtype={'SQ':str})
sel = sel.rename(columns={'SQ':'SSSQQQ'})
sel = df_B.merge(sel, on='SSSQQQ', how='left')

In [ ]:
dic_periodos = {
    '2000-2010':'00a10',
    '2010-2022':'10a22'
}

cols = ['2000-2010_EZEIS_alvara',
        '2010-2022_EZEIS_alvara',
        '2000-2010_HIS_alvara',
        '2010-2022_HIS_alvara',
        '2000-2010_HMP_alvara',
        '2010-2022_HMP_alvara',
        '2000-2010_R1_alvara',
        '2010-2022_R1_alvara',
        '2000-2010_R2_alvara',
        '2010-2022_R2_alvara']

df_sel = sel.pivot_table(index='SSSQQQ', values=cols, aggfunc='sum').reset_index()
df_sel = df_sel.rename(columns={k:'_'.join(['alvaras'] + [k.split('_')[1]] + [dic_periodos[k.split('_')[0]]]) for k in cols})

In [ ]:
df_sel = df_sel.merge(df_B, on='SSSQQQ', how='right')
vals = [c for c in df_sel.columns if c not in ['SSSQQQ', 'CD_PICSQ']]
df_sel = df_sel.pivot_table(index='CD_PICSQ', values=vals, aggfunc='sum').reset_index()

In [ ]:
df_sel.to_csv(f'{nome_dir}/alvaras.csv', index=False, sep='\t', decimal=',')

## 4. Agregação com eixos

In [ ]:
# Obter CD_PCSQ para setores de 2022
df1 = pd.read_csv('2010-2022-RMSP-RMBS-AUJ/matriz_compat_2022.csv', sep='\t', dtype={'CD_PERIMETRO':str, 'CD_GEOCODI':str})
df_22SQ = df_A.merge(df1, on='CD_PERIMETRO', how='left')[['CD_GEOCODI', 'CD_PICSQ']]

In [ ]:
# Importação dos eixos
eixos = gpd.read_file('../ATT_Censo2022_26eixos.gpkg')
#eixos = eixos[eixos['remover']!=True]
eixos = eixos[['CD_SETOR','NOME_EIXO','NOME_TRECHO_EIXO']]
eixos['CD_SETOR'] = eixos['CD_SETOR'].apply(lambda x: x.replace('P',''))
eixos = eixos.rename(columns={'CD_SETOR':'CD_GEOCODI'})

In [ ]:
# Atribuição de eixo para CD_PCSQ
df_eixos = eixos.merge(df_22SQ, on='CD_GEOCODI', how='right')[['CD_PICSQ','NOME_TRECHO_EIXO']].dropna().drop_duplicates()
dic_trechos = {row['NOME_TRECHO_EIXO']:row['NOME_EIXO'] for i, row in eixos.iterrows()}

df_eixos = df_eixos.pivot_table(index='CD_PICSQ', values='NOME_TRECHO_EIXO', aggfunc=lambda x: sorted(list(x))[0]).reset_index()
df_eixos['NOME_EIXO'] = df_eixos['NOME_TRECHO_EIXO'].apply(lambda x: dic_trechos[x])

In [ ]:
# Exportação
df_eixos.to_csv(f'{nome_dir}/eixos.csv', index=False, sep='\t', decimal=',')

## 5. Junção

In [5]:
agg_gdf = gpd.read_file('PIC1022-SSSQQQ-MSP/perimetros_compativeis_agg.gpkg')

# Mesclagem
for file in ['iptu', 'eixos', 'alvaras']:
    df_temp = pd.read_csv(f'{nome_dir}/{file}.csv', sep='\t', decimal=',', dtype={'CD_PICSQ':str})
    agg_gdf = agg_gdf.merge(df_temp, on='CD_PICSQ', how='left')

In [6]:
agg_gdf = agg_gdf.rename(columns={k:k.upper() for k in agg_gdf.columns if k!='geometry'})

In [ ]:
agg_gdf = agg_gdf.drop_duplicates(subset='CD_PICSQ')
agg_gdf.to_file(f'{nome_dir}/PICs_MSP_1022_agg.gpkg', layer='PICs_MSP_1022', driver='GPKG')